In [2]:
"""
Knowledge-Based Matchmaking System
---------------------------------
This system matches user profiles based on weighted compatibility scores across multiple attributes.
The KBS maintains profiles and uses domain-specific rules (weights, similarity metrics) to determine matches.
"""

class KnowledgeBase:
    """
    Stores and manages user profiles with unique identifiers.
    Acts as the knowledge repository for the matchmaking system.
    """
    def __init__(self):
        self.profiles = {}  # {profile_id: profile_data}
        self.next_id = 1     # Auto-incrementing ID counter

    def add_profile(self, profile_data):
        """
        Adds a new profile to the knowledge base with automatic ID assignment
        :param profile_data: Dictionary containing profile attributes
        :return: Assigned profile ID
        """
        profile_id = self.next_id
        self.profiles[profile_id] = {'id': profile_id, **profile_data}
        self.next_id += 1
        return profile_id

    def remove_profile(self, profile_id):
        """Removes a profile from the knowledge base"""
        if profile_id in self.profiles:
            del self.profiles[profile_id]

    def get_profile(self, profile_id):
        """Retrieves a profile by ID"""
        return self.profiles.get(profile_id)

    def get_all_profiles(self):
        """Returns list of all profiles for matching"""
        return list(self.profiles.values())

    def display_profiles(self):
        """Displays all available profiles with their details (excluding interests)"""
        print("\nAvailable Profiles:")
        for profile in self.profiles.values():
            print(f"ID: {profile['id']}, Name: {profile['name']}, Age: {profile['age']}, "
                  f"Gender: {profile['gender']}, Location: {profile['location']}")


class Matchmaker:
    """
    Implements matchmaking logic using compatibility rules and weights
    Leverages the knowledge base to find optimal matches
    """
    def __init__(self, knowledge_base, weights=None, max_age_diff=20):
        """
        :param knowledge_base: KnowledgeBase instance
        :param weights: Dict of attribute weights (sum to 1)
        :param max_age_diff: Maximum age difference for normalization
        """
        self.knowledge_base = knowledge_base
        # Default weights for all criteria (sum to 1)
        self.weights = weights or {
            'age': 0.1,
            'interests': 0.2,
            'location': 0.1,
            'gender': 0.1,
            'height': 0.05,
            'personality': 0.1,
            'open_mindedness': 0.1,
            'religion': 0.1,
            'has_children': 0.15
        }
        self.max_age_diff = max_age_diff  # Used for age difference normalization

    def calculate_compatibility(self, profile_a, profile_b):
        """
        Computes weighted compatibility score between two profiles
        Implements domain-specific matching rules for multiple criteria.
        Returns: Compatibility score, matched interests, and matched criteria
        """
        matched_criteria = {}  # Store matched criteria for display

        # Age Compatibility (Normalized inverse difference)
        age_diff = abs(profile_a['age'] - profile_b['age'])
        age_score = max(0, 1 - (age_diff / self.max_age_diff)) * self.weights['age']
        if age_diff <= self.max_age_diff:
            matched_criteria['age'] = f"Age difference: {age_diff} years"

        # Interest Compatibility (Jaccard Index)
        interests_a = set(profile_a['interests'])
        interests_b = set(profile_b['interests'])
        intersection = interests_a & interests_b  # Common interests
        union = interests_a | interests_b        # All unique interests
        jaccard = len(intersection) / len(union) if union else 0
        interest_score = jaccard * self.weights['interests']
        if intersection:
            matched_criteria['interests'] = f"Common interests: {', '.join(intersection)}"

        # Location Compatibility (Binary match)
        location_score = (self.weights['location']
                          if profile_a['location'] == profile_b['location']
                          else 0)
        if profile_a['location'] == profile_b['location']:
            matched_criteria['location'] = f"Location: {profile_a['location']}"

        # Gender Compatibility (Binary match)
        gender_score = (self.weights['gender']
                        if profile_a['gender_preference'] == profile_b['gender']
                        else 0)
        if profile_a['gender_preference'] == profile_b['gender']:
            matched_criteria['gender'] = f"Gender preference matched: {profile_b['gender']}"

        # Height Compatibility (Normalized inverse difference)
        height_diff = abs(profile_a['height'] - profile_b['height'])
        height_score = max(0, 1 - (height_diff / 50)) * self.weights['height']  # Assume max height difference of 50cm
        if height_diff <= 50:
            matched_criteria['height'] = f"Height difference: {height_diff} cm"

        # Personality Compatibility (Binary match)
        personality_score = (self.weights['personality']
                             if profile_a['personality'] == profile_b['personality']
                             else 0)
        if profile_a['personality'] == profile_b['personality']:
            matched_criteria['personality'] = f"Personality matched: {profile_b['personality']}"

        # Open-Mindedness Compatibility (Binary match)
        open_mindedness_score = (self.weights['open_mindedness']
                                 if profile_a['open_minded'] == profile_b['open_minded']
                                 else 0)
        if profile_a['open_minded'] == profile_b['open_minded']:
            matched_criteria['open_mindedness'] = f"Open-mindedness matched: {profile_b['open_minded']}"

        # Religion Compatibility (Binary match)
        religion_score = (self.weights['religion']
                          if profile_a['religion'] == profile_b['religion']
                          else 0)
        if profile_a['religion'] == profile_b['religion']:
            matched_criteria['religion'] = f"Religion matched: {profile_b['religion']}"

        # Children Compatibility (Binary match)
        children_score = (self.weights['has_children']
                          if profile_a['has_children'] == profile_b['has_children']
                          else 0)
        if profile_a['has_children'] == profile_b['has_children']:
            matched_criteria['has_children'] = f"Children preference matched: {profile_b['has_children']}"

        # Total Compatibility Score
        total_score = (
            age_score + interest_score + location_score + gender_score +
            height_score + personality_score + open_mindedness_score +
            religion_score + children_score
        )
        return total_score, matched_criteria

    def find_matches(self, user_id, top_n=5):
        """
        Finds top N matches for a given user based on compatibility scores
        :param user_id: Target user to find matches for
        :param top_n: Number of matches to return
        :return: List of (profile_id, score, matched_criteria) tuples sorted by score
        """
        target_profile = self.knowledge_base.get_profile(user_id)
        if not target_profile:
            return []

        matches = []
        for profile in self.knowledge_base.get_all_profiles():
            if profile['id'] == user_id:
                continue  # Skip self-comparison

            score, matched_criteria = self.calculate_compatibility(target_profile, profile)
            matches.append((profile['id'], round(score, 2), matched_criteria))

        # Sort by descending score and return top N results
        matches.sort(key=lambda x: x[1], reverse=True)
        return matches[:top_n]


def main():
    # Initialize Knowledge Base
    kb = KnowledgeBase()

    # Populate with 6 sample profiles
    profiles = [
        {
            'name': 'Alice', 'age': 25, 'gender': 'Female', 'gender_preference': 'Male',
            'height': 165, 'personality': 'Extroverted', 'open_minded': True,
            'religion': 'Christian', 'has_children': False,
            'interests': ['hiking', 'cooking', 'movies'], 'location': 'New York'
        },
        {
            'name': 'Bob', 'age': 28, 'gender': 'Male', 'gender_preference': 'Female',
            'height': 180, 'personality': 'Introverted', 'open_minded': True,
            'religion': 'Agnostic', 'has_children': False,
            'interests': ['cooking', 'travel', 'photography'], 'location': 'New York'
        },
        {
            'name': 'Charlie', 'age': 35, 'gender': 'Male', 'gender_preference': 'Female',
            'height': 175, 'personality': 'Extroverted', 'open_minded': False,
            'religion': 'Atheist', 'has_children': True,
            'interests': ['gaming', 'coding', 'reading'], 'location': 'Chicago'
        },
        {
            'name': 'Diana', 'age': 40, 'gender': 'Female', 'gender_preference': 'Male',
            'height': 160, 'personality': 'Introverted', 'open_minded': True,
            'religion': 'Christian', 'has_children': True,
            'interests': ['hiking', 'travel', 'photography'], 'location': 'Boston'
        },
        {
            'name': 'Eve', 'age': 30, 'gender': 'Female', 'gender_preference': 'Male',
            'height': 170, 'personality': 'Extroverted', 'open_minded': True,
            'religion': 'Christian', 'has_children': False,
            'interests': ['yoga', 'reading', 'travel'], 'location': 'New York'
        },
        {
            'name': 'Frank', 'age': 32, 'gender': 'Male', 'gender_preference': 'Female',
            'height': 178, 'personality': 'Introverted', 'open_minded': False,
            'religion': 'Agnostic', 'has_children': False,
            'interests': ['photography', 'movies', 'coding'], 'location': 'Chicago'
        },
    ]
    for p in profiles:
        kb.add_profile(p)

    # Initialize Matchmaker with custom weights
    matchmaker = Matchmaker(
        knowledge_base=kb,
        weights={
            'age': 0.1,
            'interests': 0.2,
            'location': 0.1,
            'gender': 0.1,
            'height': 0.05,
            'personality': 0.1,
            'open_mindedness': 0.1,
            'religion': 0.1,
            'has_children': 0.15
        },
        max_age_diff=25
    )

    # Display available profiles (without interests)
    kb.display_profiles()

    # Allow user to select a profile
    try:
        selected_id = int(input("\nEnter the ID of the profile you want to match: "))
        selected_profile = kb.get_profile(selected_id)
        if not selected_profile:
            print("Invalid profile ID. Please try again.")
            return

        # Display compatibility scores and matched criteria for the selected profile
        print(f"\nCompatibility Scores for {selected_profile['name']}:")
        matches = matchmaker.find_matches(selected_id, top_n=len(kb.get_all_profiles()) - 1)
        for match in matches:
            profile = kb.get_profile(match[0])
            print(f"\nProfile {profile['name']} (ID: {profile['id']}):")
            print(f"  Compatibility Score: {match[1]}")
            if match[2]:  # Show matched criteria if any
                print("  Matched Criteria:")
                for criterion, details in match[2].items():
                    print(f"    - {details}")
            else:
                print("  No matching criteria.")
    except ValueError:
        print("Invalid input. Please enter a valid profile ID.")


if __name__ == "__main__":
    main()


Available Profiles:
ID: 1, Name: Alice, Age: 25, Gender: Female, Location: New York
ID: 2, Name: Bob, Age: 28, Gender: Male, Location: New York
ID: 3, Name: Charlie, Age: 35, Gender: Male, Location: Chicago
ID: 4, Name: Diana, Age: 40, Gender: Female, Location: Boston
ID: 5, Name: Eve, Age: 30, Gender: Female, Location: New York
ID: 6, Name: Frank, Age: 32, Gender: Male, Location: Chicago

Enter the ID of the profile you want to match: 6

Compatibility Scores for Frank:

Profile Bob (ID: 2):
  Compatibility Score: 0.52
  Matched Criteria:
    - Age difference: 4 years
    - Common interests: photography
    - Height difference: 2 cm
    - Personality matched: Introverted
    - Religion matched: Agnostic
    - Children preference matched: False

Profile Alice (ID: 1):
  Compatibility Score: 0.4
  Matched Criteria:
    - Age difference: 7 years
    - Common interests: movies
    - Gender preference matched: Female
    - Height difference: 13 cm
    - Children preference matched: False

